In [249]:
%%capture
%run 02_account.ipynb

In [250]:
from objsize import get_deep_size as get_size

## Block

In [299]:
class Block: 
    def __init__(self, txs, prev_hash, number):
        self.txs       = self.val_txs(txs)
        self.prev_hash = prev_hash
        self.number    = number
        self.time      = time.time()
        self.mined     = False
            
    def add(self, tx, is_coinbase=False):
        assert          val_tx(tx), 'tx validation failed'
        assert not self.is_dup(tx), 'tx is duplicated'
        if is_coinbase: self.txs.insert(0, tx)
        else          : self.txs.append(tx)
            
    def is_dup(self, tx):
        tx_hashes = [tx.hash for tx in self.txs]
        return tx.hash in tx_hashes
    
    def val_txs(self, txs):
        if False not in [val_tx(tx) for tx in txs]: return txs
        else: raise Exception('TXs validation failed')
        
    def __str__(self): 
        block_str = ('time:\t\t'        + time.ctime(self.time)+
                     '\nnumber:\t\t'    + str(self.number)+
                     '\nprev_hash:\t'   + ph(self.prev_hash)+
                     '\n\ntxs:\t\t\n\n' + txs2str(self.txs))
        if self.mined: 
            block_str += ('\ndiff:\t\t'  +str(self.diff)+
                          '\nreward:\t\t'+str(self.reward)+
                          '\nnonce:\t\t' +str(self.nonce)) 
        return block_str
    def __bytes__(self): return (txs2str(self.txs)+
                                 self.prev_hash+
                                 str(self.number)+
                                 str(self.time)).encode()

In [300]:
acc1,acc2 = Account(),Account()

In [301]:
tx1 = acc1.signed_tx(acc2, 24)
tx2 = acc2.signed_tx(acc1, 18)

block = Block([tx1,tx2], rh(), 2); print(block)

time:		Fri Mar 26 22:37:27 2021
number:		2
prev_hash:	💴 4e609288a5c16fa4...ca4

txs:		

time:	Fri Mar 26 22:37:27 2021
from:	💽 0x57E68B006B350e...8F1
to:	📺 0x944a5b9D81Baee...08b
value:	24 ether
nonce:	0
hash:	🕖 f05585434a090f1c...d74
signed:	true

time:	Fri Mar 26 22:37:27 2021
from:	📺 0x944a5b9D81Baee...08b
to:	💽 0x57E68B006B350e...8F1
value:	18 ether
nonce:	0
hash:	🕍 e7335f67e21f07d6...e60
signed:	true



If anything in the tx is changed, like increasing the value to send, the signature should become invalid and the block creation throws an exception.

In [302]:
tx2.value = 1800

val_failed = False
try:    Block([tx1,tx2], rh(), 2, time.time())
except: val_failed = True
assert  val_failed

## Mining

Mining is a crucial component on any blockchain relying on proof of work (abbr. pow). It is used to reach consensus in a network of anonymous decentralized nodes.

Pow uses computational ressources to calculate a specific hash. This can only be done by brute force. 

In [307]:
class Miner:
    def __init__(self, acc): 
        self.acc = acc
        self.pub = acc.pub
        
    def coinbase(self, reward): 
        return self.acc.sign(TX(self.pub, self.pub, reward, self.acc.nonce))
    
    def mine(self, block, diff, reward, attempts=1000):
        mb = deepcopy(block)
        mb.add(self.acc.sign(self.coinbase(reward)), is_coinbase=True)
        mb_b = bytes(mb)
        nonce = 0
        for i in range(attempts):
            candidate   = mb_b + str(nonce).encode()
            candidate_h = sha(candidate)
            if candidate_h[:diff] == '0'*diff: break
            nonce += 1
        mb.diff   = diff
        mb.reward = reward
        mb.nonce  = nonce
        mb.mined  = True
        return mb
    
    def mine_genesis(self, txs):
        block = Block(txs, '0x0', 0)
        return self.mine(block, 1, 100)

In [308]:
miner = Miner(acc1)
mb = miner.mine(block, 2, 100); print(mb)

time:		Fri Mar 26 22:37:27 2021
number:		2
prev_hash:	💴 4e609288a5c16fa4...ca4

txs:		

time:	Fri Mar 26 22:45:48 2021
from:	💽 0x57E68B006B350e...8F1
to:	💽 0x57E68B006B350e...8F1
value:	100 ether
nonce:	5
hash:	🔻 d5d5c8f86af25e7a...3bd
signed:	true

time:	Fri Mar 26 22:37:27 2021
from:	💽 0x57E68B006B350e...8F1
to:	📺 0x944a5b9D81Baee...08b
value:	24 ether
nonce:	0
hash:	🕖 f05585434a090f1c...d74
signed:	true

time:	Fri Mar 26 22:37:27 2021
from:	📺 0x944a5b9D81Baee...08b
to:	💽 0x57E68B006B350e...8F1
value:	1800 ether
nonce:	0
hash:	🔇 a1d0dfdf75e6b59c...12d
signed:	true

diff:		2
reward:		100
nonce:		191


In [309]:
assert len(mb.txs) == 3

### Genesis Block

Is the first block in the blockchain. This is how it all begins.

In [312]:
gb = miner.mine_genesis([tx1])
print(gb)

time:		Fri Mar 26 22:46:35 2021
number:		0
prev_hash:	0x0

txs:		

time:	Fri Mar 26 22:46:35 2021
from:	💽 0x57E68B006B350e...8F1
to:	💽 0x57E68B006B350e...8F1
value:	100 ether
nonce:	11
hash:	📋 65ea22e15da7ab67...3bc
signed:	true

time:	Fri Mar 26 22:37:27 2021
from:	💽 0x57E68B006B350e...8F1
to:	📺 0x944a5b9D81Baee...08b
value:	24 ether
nonce:	0
hash:	🕖 f05585434a090f1c...d74
signed:	true

diff:		1
reward:		100
nonce:		15
